In [1]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString
from ST import printSymTab

def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i)
    def writeln(s): print('\n')
    def read(s): return int(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})

Importing Jupyter notebook from P0.ipynb
Importing Jupyter notebook from SC.ipynb
Importing Jupyter notebook from ST.ipynb


### Scanner Tests for Exception

In [2]:
import SC
def scanString(src):
    SC.init(src); syms = []
    while SC.sym != SC.EOF:
        syms.append(('INDENT' if SC.sym == SC.INDENT else \
                     'DEDENT' if SC.sym == SC.DEDENT else \
                     'IDENT' if SC.sym == SC.IDENT else SC.sym, SC.newline))
        SC.getSym()
    return syms

In [ ]:
scanString("""

program p

  if a then
    writeln()
  else
    writeln()
  if a then writeln() else writeln()
""")

### Compiler Tests for Exception

In [4]:
# "Throw" can be compiled，but the WebAssembly Code is incorrect
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw else r := x
program equationsolver
    var a: integer
        a ← sqrt(2)
        write(a)
""")

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(func $sqrt (param $x i32) (result i32)
(local $r i32)
(local $0 i32)
local.get $x
i32.const 0
i32.lt_s
if
throw 0
else
local.get $x
local.set $r
end
local.get $r)
(func $program
(local $a i32)
(local $0 i32)
i32.const 2
call $sqrt
local.set $a
local.get $a
call $write
)
(memory 1)
(start $program)
)


In [10]:
!wat2wasm test.wat

test.wat:12:1: error: opcode not allowed: throw
throw 0
^^^^^


In [ ]:
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then r := 0 else r := x
program equationsolver
    var a: integer
        a ← sqrt(2)
        write(a)
""")

In [ ]:
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw else ...
procedure quadraticsolution(a, b, c: integer) → (x, y: integer)
    var d: integer
        d ← sqrt(a × a - 4 × a × c)
        x, y := (- b + d) div (2 × a), (- b - d) div (2 × a)
program equationsolver
    var a, b, c: integer; done: boolean
        done := false
        while ¬done do
            try
                a ← read(); b ← read(); c ← read()
                x, y ← quadraticsolution(a, b, c)
                write(x); write(y)
           catch done := true
""")